## Question 1
To be able to use this model, we need to know the name of the input and output nodes.

What's the name of the output:

- `output`
- `sigmoid`
- `softmax`
- `prediction`

In [12]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 23.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 25.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.2/323.2 kB 8.3 MB/s eta 0:00:00:00:01

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [1]:
import onnx

# 讀取 ONNX 模型
model = onnx.load("hair_classifier_v1.onnx")

# 列出所有輸入節點
print("Inputs:")
for i in model.graph.input:
    print(i.name)

# 列出所有輸出節點
print("\nOutputs:")
for o in model.graph.output:
    print(o.name)


Inputs:
input

Outputs:
output


### Preparing the image
You'll need some code for downloading and resizing images. You can use this code:

In [2]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [10]:
!pip install pillow


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


## Question 2: Target size
Let's download and resize this image:

`https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg`

Based on the previous homework, what should be the target size for the image?

- 64x64
- 128x128
- 200x200
- 256x256


## Question 3
Now we need to turn the image into numpy array and pre-process it.

Tip: Check the previous homework. What was the pre-processing we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

- -10.73
- -1.073
- 1.073
- 10.73

In [3]:
img = download_image('https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg')

In [4]:
import numpy as np
# Resize
img = img.resize((200, 200), Image.NEAREST)

# 轉 numpy array 並縮放
img_arr = np.array(img).astype(np.float32) / 255.0

# ImageNet normalization
mean = np.array([0.485, 0.456, 0.406])
std  = np.array([0.229, 0.224, 0.225])
img_arr = (img_arr - mean) / std

# 查看第一個像素 R channel
first_pixel_r = img_arr[0,0,0]
print(first_pixel_r)

-1.0732939435925546


## Question 4
Now let's apply this model to this image. What's the output of the model
- 0.09
- 0.49
- 0.69
- 0.89

In [35]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 10.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [6]:
import onnxruntime as ort
import numpy as np
from torchvision import transforms


# 定義 preprocessing
preprocess = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

img_tensor = preprocess(img)  # [3, 200, 200]
img_tensor = img_tensor.unsqueeze(0).numpy()  # [1, 3, 200, 200]，onnxruntime 需要 numpy

# 載入模型
session = ort.InferenceSession("hair_classifier_v1.onnx")
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

# 推論
pred = session.run([output_name], {input_name: img_tensor})[0]
print(pred)


[[0.09156641]]


## uestion 5
Download the base image `agrigorev/model-2025-hairstyle:v1`. You can do it with docker pull.

So what's the size of this base image?

- 88 Mb
- 208 Mb
- 608 Mb
- 1208 Mb
You can get this information when running docker images - it'll be in the "SIZE" column.